**Caso de Estudio**
## Machine Learning Avanzado
## Proyecto: Outbrain Click Prediction

Integrantes: Carlos Bustamante, Nicolás Rivera, Pablo Elgueta y Patricio Ramirez.


---



El caso seleccionado es un desafío de Kaggle del año 2017, donde se busca predecir el contenido que una persona cliquearía en distintas páginas web. El desafío [Outbrain Click Prediction](https://www.kaggle.com/competitions/outbrain-click-prediction/overview) sorteó USD$25.000 en premios para los tres mejores lugares y contó con una amplia participación durante el concurso.

El dataset contine muestras de vistas y clicks de usuarios en Estados Unidos observados durante el 2016. Cada página web o aviso publicitario clickeado (display_id) contiene además información de sus características. También hay información de recomendaciones dadas a usuarios específicos en contextos específicos.

Para su resolución se seguiran las siguientes etapas:

*   Reconocimiento e importación de las librerías y módulos utilizados
*   Procesamiento de datos
*   Exploración descriptiva 
*   Aplicación de un modelo de factorización de matrices
*   Aplicación de un modelo de factorización de máquinas
*   Comparación, discusión y conclusiones

---




# Importación de librerías:

In [1]:
from __future__ import print_function, division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import os
#import chardet
from collections import Counter
#from google.colab import drive
#drive.mount("/content/drive")
#path = '/content/drive/My Drive/MG Data Science/MLA/Tarea 3/'

### Ahora en Keras

from builtins import range, input
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import tensorflow
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.model_selection import train_test_split

from collections import Counter

# Procesamiento de datos y exploración descriptiva:

**Resumen de los campos:**

**display_id**: Identificación del contexto o situación en la que un usuario clickeó un aviso.

**ad_id**: Identificación del aviso publicitario.

**clicked**: Columna binaria que simboliza si el usiario clickeó o no.

**document_id**: Identificación de una página web

**category_id**: Identificación del tipo de contenido de un ad.

**confidence_level**: Confianza que tiene la empresa proveedora entre una relación.

**campaing_id**: Identificación de la campaña a la cual pertenece un ad.

**advertiser_id**: Identificación de la empresa a la cual pertenece un ad.

**uuid**: Identificación de un usuario.

**timestamp**: Fecha de ejecución.

**platform**: Si es un (1) desktop, (2) celular o (3) laptop.

**geolocation**: País>Estado>DMA

**topic_id**: Identificación del tópico del aviso.

**traffic_source**: (1) interno, (2) búsqueda o (3) social.

**entity_id**: Identificación de una persona, organización o lugar.

**publisher_id**: Identificación del editorial.

**publish_time**: Momento de una publicación.


###Train

In [2]:
df_clicks_train = pd.read_csv('clicks_train.csv')
df_clicks_train

,display_id,ad_id,clicked
0,1,42337,0
1,1,139684,0
2,1,144739,1
3,1,156824,0
4,1,279295,0
...,...,...,...
87141726,16874592,186600,0
87141727,16874593,151498,1
87141728,16874593,282350,0
87141729,16874593,521828,0


## Test

In [4]:
df_clicks_test = pd.read_csv('clicks_test.csv')
df_clicks_test

FileNotFoundError: [Errno 2] No such file or directory: 'clicks_test.csv'

## Documents Categories

In [ ]:
df_documents_categories = pd.read_csv('Tarea3/documents_categories.csv')
df_documents_categories

## Sample Submission

In [ ]:
df_sample_sub = pd.read_csv('Tarea3/sample_submission.csv')
df_sample_sub

## Promoted Content

In [ ]:
df_promoted_cont = pd.read_csv('Tarea3/promoted_content.csv')
df_promoted_cont

## Events

In [ ]:
df_events = pd.read_csv('Tarea3/events.csv')
df_events = df_events[['uuid','display_id','document_id','timestamp','platform','geo_location']]
df_events

Hay usuarios que tienen más de un display.

In [ ]:
df_documents_topics = pd.read_csv('Tarea3/documents_topics.csv')
df_documents_topics

## Page Views Sample

## Document Topics

In [ ]:
df_page_view_sample = pd.read_csv('Tarea3/page_views_sample.csv')
df_page_view_sample

## Document Entities

In [ ]:
df_documents_entities = pd.read_csv('Tarea3/documents_entities.csv')
df_documents_entities

## Documents Meta

In [ ]:
df_documents_meta = pd.read_csv('Tarea3/documents_meta.csv')
df_documents_meta

merge

# Exploración descriptiva:

Dentro del dataset, la base que se utilizó para creación del modelo predictivo con factorización de matrices es "click_train.csv". Esta data cuenta con 87.141.731 filas y 3 columnas.

La cantidad de ads máxima que un usuario clickeó dentro de la base es 12, mientras que el mínimo es 2, siendo 4 el número más común de ads clickeados, seguido de 6.

In [ ]:
ad_in_display = df_clicks_train.groupby('display_id')['ad_id'].count().value_counts()
sns.barplot(ad_in_display.index, ad_in_display.values)

También es posible extraer información valiosa de otras bases de datos contenidas en este desafío, por ejemplo, es posible graficar la cantidad de veces que aparecen publicados los ads en las distintas plataformas. Un 61,7% de los ads aparece hasta 10 veces.

In [ ]:
ad_usage_train = df_clicks_train.groupby('ad_id')['ad_id'].count()

for i in [2, 10, 50, 100, 1000]:
    print('Ads that appear less than {} times: {}%'.format(i, round((ad_usage_train < i).mean() * 100, 2)))

plt.figure(figsize=(12, 6))
plt.hist(ad_usage_train.values, bins=50, log=True)
plt.xlabel('Number of times ad appeared', fontsize=12)
plt.ylabel('log(Count of displays with ad)', fontsize=12)
plt.show()

# Modelo de Factorización de Matrices

Primero, queremos dejar como valores categoricos a las dos columnas las cuales vamos a utilizar para el entrenamiento, que en este caso son las de display_id y ad_id.

In [ ]:
df_clicks_train.display_id = df_clicks_train.display_id.astype('category').cat.codes.values
df_clicks_train.ad_id = df_clicks_train.ad_id.astype('category').cat.codes.values

Despues, ya que esta dataframe contiene muchisimos datos (no tenemos la capacidad computacional para poder procesar tantos datos) necesitamos hacerle una reduccion, y para hacer esto ocupamos la libreria Counter para buscar los usuarios y anuncios que mas se repiten en la data. Al hacer esto, podemos reducir los datos sin perder tanta precision ya que estamos utilizando los datos mas comunes.

In [ ]:
n = 10000
m = 8000

from collections import Counter

In [ ]:
ucount = Counter(df_clicks_train['display_id'])
mcount = Counter(df_clicks_train['ad_id'])

uid = [u for u, c in ucount.most_common(n)]
mid = [u for u, c in mcount.most_common(m)]

In [ ]:
newdf = df_clicks_train[df_clicks_train['display_id'].isin(uid) & df_clicks_train['ad_id'].isin(mid)]
newdf.head()

Hacemos la misma transformacion de datos para la nueva dataframe

In [ ]:
newdf.display_id = newdf.display_id.astype('category').cat.codes.values
newdf.ad_id = newdf.ad_id.astype('category').cat.codes.values

In [ ]:
train, test = train_test_split(newdf, test_size=0.2)

In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [ ]:
n_users, n_ads = 10000 , 8000
n_latent_factors = 20

En la fase de entrenamiento, al estar tratando con una factorizacion de matrices, debemos utilizar el producto punto entre las matrices de anuncios y personas, por lo que para eso hacemos un embedding y despues un flatten. Al final, podemos hacerle el producto punto a las matrices flatten de usuarios y anuncios

In [ ]:
ad_input = keras.layers.Input(shape=[1],name='Item')
ad_embedding = keras.layers.Embedding(n_ads + 1, n_latent_factors, name='Ad-Embedding')(ad_input)
ad_vec = keras.layers.Flatten(name='FlattenAds')(ad_embedding)
user_input = keras.layers.Input(shape=[1],name='User')
user_vec = keras.layers.Flatten(name='FlattenUsers')(keras.layers.Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
prod = keras.layers.dot([ad_vec, user_vec], axes=1,name='DotProduct')
print(prod)
model = keras.Model([user_input, ad_input], prod)

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mse'])

In [ ]:
model.summary()

In [ ]:
history = model.fit([train.display_id, train.ad_id], train.clicked, epochs=25, verbose=0, 
                    validation_data = ([test.display_id, test.ad_id], test.clicked))

Aqui podemos observar el tema de overfitting que existe dentro de los modelos de factorizacion de matrices y sus derivados. Dado que estos modelos no son generalizables, estos se ajustan mucho a los valores actuales y lo que tiende a pasar es que para los datos de entrenamiento existe perdida pero cuando entramos a ver los datos de validacion estos tienden a subir, o a bajar y subir. En este caso, los datos de validacion suben mientras que los de entrenamiento bajan, y despues ambos bajan con la misma pendiente.

In [ ]:
pd.Series(history.history['loss']).plot(logy=True, color='b')
plt.xlabel("Epoch")
plt.ylabel("Training Error")

In [ ]:
pd.Series(history.history['val_loss']).plot(logy=True, color='orange')
plt.xlabel("Epoch")
plt.ylabel("Validation Error")

In [ ]:
results = model.evaluate((test.display_id, test.ad_id), test.clicked, batch_size=1)

# Predicciones Factorizacion de Matrices

**Finalmente, como queremos hacer predicciones, vamos a predecir los primeros cinco anuncios que se le podrian dar a una persona dado que ha visto anuncios anteriormente.** 

In [ ]:
ad_embedding_learnt = model.get_layer(name='Ad-Embedding').get_weights()[0]
pd.DataFrame(ad_embedding_learnt).describe()

In [ ]:
user_embedding_learnt = model.get_layer(name='User-Embedding').get_weights()[0]

In [ ]:
def recommend(user_id, number_of_ads=5):
    ads = user_embedding_learnt[user_id]@ad_embedding_learnt.T
    mids = np.argpartition(ads, -number_of_ads)[-number_of_ads:]
    return mids

**Estos son los 5 ad_id que le estariamos recomendando a nuestro  primer usuario**

In [ ]:
recommend(user_id=1)

# Modelo de Factorizacion de Maquinas

Comenzamos creando una clase la cual va a contener todas las columnas necesarias para ejecutar el modelo, ademas de eso creamos las caracteristicas necesarias para el entrenamiento posterior, como la semilla, las epocas, y el valor de regularizacion.

In [ ]:
class Config:
    category_col = ['display_id', 'ad_id', 'uuid', 'document_id',
       'campaign_id', 'advertiser_id']
    num_col = []
    target_col = ['clicked']
    
    seed=2021
    epochs=5
    batch_size=128
    seed=17
    embedding_dim=8
    lr=1e-4
    
config=Config()

In [ ]:
data_df = df_clicks_train
item_df = df_promoted_cont
user_df = df_events

Hacemos un drop a las columnas de localizacion dado a que no nos ofrece informacion inmediata para procesarla en el modelo, el document id por temas de incongruencias con la dataframe de items, el cual ya trae un document id. El timestamp, el cual solo trae informacion de tiempo y no es tan necesario procesarla en este modelo. Y por ultimo, el platform, el cual trae problemas al hacer preprocesamiento, dado que trae dentro de el datos vacios los cuales no fuimos capaces de rellenar, dando problemas al hacer la transformacion de datos.

In [ ]:
user_df = user_df.drop(columns=['geo_location', 'document_id', 'timestamp', 'platform'])

Por ultimo, juntamos los datos para poder procesarlos


Es de notar para este ejercicio las pocas columnas que estamos utilizando, esto es dado a la gran cantidad de datos que ofrece el ejercicio, ya que, para los datos de documentacion, al juntarlos con los datos principales llegamos a una cantidad de datos de aproximadamente 1700 mil millones los cuales consumen mucha memoria y para nosotros no es posible juntarlos, por lo que por temas de memoria y simplicidad decidimos sacarlos.

In [ ]:
def merge_df(data_df, item_df, user_df):
    tmp = pd.merge(data_df, user_df, on='display_id', how='inner')
    tmp = pd.merge(tmp, item_df, on='ad_id', how='inner')
    tmp = tmp
    return tmp

df = merge_df(data_df, item_df, user_df)
df

Ya despues de haber juntado los datos, procedemos a crear dos clases, una la cual construye las funciones para el preprocesamiento, y otra la cual hace el preprocesamiento. En la primera, utilizamos la clase creada al comienzo para pasarle una funcion Pipeline, la cual nos da el orden en el cual se va a ejecutar el modelo, llenando los datos numericos con un promedio de los demas datos, y datos NaN para las columnas categoricas. Despues, se le hace una transformacion con la funcion ColumnTransformer a los datos numericos y categoricos. Como es de notar en nuestro caso, nostros no vamos a utilizar datos numericos, por lo cual las funciones para los datos numericos no son de utilidad para nosotros en este momento. 

Despues, la funcion de preprocesamiento hace la reduccion de dimensiones al pasarle como argumento cuantas veces aparece un anuncio dentro del dataframe y por ultimo utilizando la funcion anterior genera el pipeline.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


def build_preprocessor(config): 
    category_col = config.category_col
    num_col = config.num_col
    
    num_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ('std', (StandardScaler())),])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="constant", fill_value='NAN')),
        ('oe', (OrdinalEncoder())),
        ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', num_transformer, num_col),
            ('cat', categorical_transformer, category_col),
        ],
        remainder="drop")
    return preprocessor
    
def preprocess(df, config):
    
    category_col = config.category_col
    num_col = config.num_col
    target_col = config.target_col
    
    print(target_col)
    
    print(df.shape)
    ad_cnt = df.groupby('ad_id').size()
    use_ad = list(ad_cnt[ad_cnt > 210000].index)
    df = df[df['ad_id'].isin(use_ad)]
    print(df.shape)

    # Extract release year.
    #df["year"] = df['release_date'].apply(lambda x: str(x).split('-')[-1]) 

    # Create a label column for binary classification.
    #df.insert(df.shape[1],target_col,df['clicked'],True)
    #df[target_col] = df['clicked'] >= 4.0
    #df[target_col] = df[target_col].astype(int)

    # Build pipeline
    pp = build_preprocessor(config)
    pp.fit(df)
    return df, pp

In [ ]:
df, pp = preprocess(df, config)

In [ ]:
pp.transform(df).shape

Comenzamos el entrenamiento haciendo la separacion de datos

In [ ]:
# split data
tra_df, val_df = train_test_split(df, test_size=0.2, stratify=df['ad_id'], random_state=config.seed)
print(tra_df.shape)
print(val_df.shape)

En la construccion del modelo, hay que tomar en cuenta la adicion de features al modelo de la factorizacion de matrices lo cual nos deja con muchos multiplicaciones por hacer. Es por esto que despues de hacerle el embedding a todas las columnas, las cuales nos ayudan a procesar datos grandes, le hacemos un flattening para despues proceder a hacer el producto punto y por ultimo la adicion de todas esas multiplicaciones para llegar al resultado del modelo.

In [ ]:
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, add, Activation, dot
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2 as l2_reg
import itertools
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping

def build_model(category_num, category_cols, num_cols, K=20, solver='adam', l2=0, l2_fm=1e-3):

    # Numerical features
    num_inputs = [Input(shape=(1,), name=col,) for col in num_cols]
    # Categorical features
    cat_inputs = [Input(shape=(1,), name=col,) for col in category_cols]

    inputs = num_inputs + cat_inputs

    flatten_layers=[]
    # Numerical featrue embedding
    for enc_inp, col in zip(num_inputs, num_cols):
        # num featrue dence
        x = Dense(K, name = f'embed_{col}',kernel_regularizer=l2_reg(l2_fm))(enc_inp)
        flatten_layers.append(x)

    # Category feature embedding
    for enc_inp, col in zip(cat_inputs, category_cols):
        num_c = category_num[col]
        embed_c = Embedding(input_dim=num_c,
                            output_dim=K,
                            input_length=1,
                            name=f'embed_{col}',
                            embeddings_regularizer=l2_reg(l2_fm))(enc_inp)
        flatten_c = Flatten()(embed_c)
        flatten_layers.append(flatten_c)
                
    # Feature interaction term
    fm_layers = []
    for emb1,emb2 in itertools.combinations(flatten_layers, 2):
        dot_layer = dot([emb1,emb2], axes=1)
        fm_layers.append(dot_layer)
    #print(fm_layers)
        

    # Linear term
    for enc_inp,col in zip(cat_inputs, category_cols):
        # embedding
        num_c = category_num[col]
        embed_c = Embedding(input_dim=num_c,
                            output_dim=1,
                            input_length=1,
                            name=f'linear_{col}',
                            embeddings_regularizer=l2_reg(l2_fm))(enc_inp)
        flatten_c = Flatten()(embed_c)
        fm_layers.append(flatten_c)
                
    for enc_inp, col in zip(num_inputs, num_cols):
        x = Dense(1, name = f'linear_{col}',kernel_regularizer=l2_reg(l2_fm))(enc_inp)
        fm_layers.append(x)

    # Add all terms
    flatten = add(fm_layers)
    outputs = Activation('sigmoid',name='outputs')(flatten)
    
    model = Model(inputs=inputs, outputs=outputs)

    model.compile(
                optimizer=solver,
                loss='binary_crossentropy',
                metrics='accuracy'
              )

    return model 

In [ ]:
category_num = {col: df[col].nunique() for col in config.category_col}
model = build_model(category_num, config.category_col, config.num_col, K=config.embedding_dim)

In [ ]:
model.summary()

Al ver los resultados, se nota que los valores de perdida en especial en los datos de validacion son minusculos, y esto se debe al gran problema de overfitting que tienen las factorizaciones de matrices y maquinas. Para esto ultimo, utilizamos el regularizador l2 el cual nos ayuda a castigar los pesos asociados en el modelo, aproximandolos a 0 pero nunca dejandolos en 0. Esto disminuye la influencia que tienen los pesos por sobre los nodos y nos sirve para quitar el overfitting.

In [ ]:
cb = [EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=2, verbose=0,)]

feature_num = len(config.category_col + config.num_col)
tra_inputs = [pp.transform(tra_df)[:, i] for i in range(feature_num)]
val_inputs = [pp.transform(val_df)[:, i] for i in range(feature_num)]

history = model.fit(
          #x=pp.transform(tra_df).reshape(len(tra_df), feature_num, 1),
          x=tra_inputs,
          y=tra_df[config.target_col],
          epochs=config.epochs,
          batch_size=config.batch_size,
          validation_data=(val_inputs,
                           val_df[config.target_col]),
          callbacks=cb
         )

En los graficos se ve mas claro el tema del potencial overfitting que puede existir facilmente en estos modelos de FM.

In [ ]:
import matplotlib.pyplot as plt
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Valid'], loc='upper left')
    plt.show()    
plot_history(history)

Por ultimo, para poder hacer predicciones, tomamos a un usuario y predecimos si el le fuera a hacer click a un anuncio o no.

In [ ]:
user='a8adbb70abf1a5' #user_id
user_df = val_df.reset_index(drop=True).query('uuid==@user')
user_df.head()

In [ ]:
user_df.index

In [ ]:
user_inputs = [pp.transform(val_df)[user_df.index, i] for i in range(feature_num)]
user_df['pred'] = model.predict(user_inputs)
user_df = user_df.sort_values('pred', ascending=False)

In [ ]:
user_df[['ad_id','clicked','pred']].head(50)

# Conclusión

En conclusión, se puede ver que existen algunas diferencias entre la factorización de matrices y la factorización de maquinas, por lo cual existen ventajas y desventajas entre ellas: Una desventaja de la factorización de matrices, viene siendo que es no es generalizable, son especificamente construidos para un problema dado. Los algoritmos de aprendizaje e implementación son hechos a medida para modelos individuales. Una ventaja es que podemos estimar o calcular interacciones entre dos o mas variables incluso si la interaccion no ha sido observada, o sea, podemos pasarle usuarios y peliculas, y que este nos arroje un valor que ese usuario le fuera a dar a una pelicula, si es que la hubiese visto.

En el caso de la factorización de máquinas, sabemos que es un modelo derivado que utiliza la factorización de matrices, pero tiene la ventaja de que puede utilizar la regresión polinomial, por lo que esto nos ayuda a poder tomar mas variables,  llegando a un modelo mas generalizable. Una desventaja que tienen ambas es que requieren mas cálculo que otros modelos similares, y en especial el de factorización de máquinas. Mientras más columnas con datos le agreguemos, más parametros tendrá el modelo al final y más tiempo se consumirá para poder entrenar el modelo. Por lo que, el modelo de factorización de máquinas dependiendo de la capacidad computacional, podria no ser bueno dado a la poca cantidad de datos que puedes utilizar. 